In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be s

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [3]:
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, random_split, DataLoader
from PIL import Image
import torchvision.models as models
from tqdm.notebook import tqdm
import torchvision.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from torchvision.utils import make_grid

import os
import re
import requests

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [4]:
model_category = 'Women Tshirts'

In [ ]:
train_df_Men_Tshirts = pd.read_csv('../../Preprocessor-FillNA/output/train_fillna_Women Tshirts_effnet_b5_4epochs.csv')
# train_df_Men_Tshirts = train_df_Men_Tshirts.head(1000)

In [6]:
train_df_Men_Tshirts

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,image_path
0,32601,Women Tshirts,8,multicolor,loose,long,default,default,default,regular sleeves,default,/kaggle/input/visual-taxonomy/train_images/032...
1,32602,Women Tshirts,8,yellow,loose,long,default,default,long sleeves,regular sleeves,default,/kaggle/input/visual-taxonomy/train_images/032...
2,32603,Women Tshirts,8,multicolor,loose,long,default,default,default,regular sleeves,default,/kaggle/input/visual-taxonomy/train_images/032...
3,32604,Women Tshirts,8,multicolor,loose,long,default,default,long sleeves,regular sleeves,default,/kaggle/input/visual-taxonomy/train_images/032...
4,32605,Women Tshirts,8,multicolor,loose,long,default,default,default,regular sleeves,default,/kaggle/input/visual-taxonomy/train_images/032...
...,...,...,...,...,...,...,...,...,...,...,...,...
18769,51370,Women Tshirts,8,white,regular,crop,printed,funky print,short sleeves,regular sleeves,default,/kaggle/input/visual-taxonomy/train_images/051...
18770,51371,Women Tshirts,8,white,regular,crop,printed,typography,short sleeves,regular sleeves,default,/kaggle/input/visual-taxonomy/train_images/051...
18771,51372,Women Tshirts,8,yellow,regular,crop,printed,typography,short sleeves,regular sleeves,default,/kaggle/input/visual-taxonomy/train_images/051...
18772,51373,Women Tshirts,8,white,regular,regular,printed,funky print,short sleeves,regular sleeves,default,/kaggle/input/visual-taxonomy/train_images/051...


In [7]:
attr_columns = train_df_Men_Tshirts.filter(like='attr_').columns.to_list() # Adjust if more attributes exist
print(attr_columns)

['attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5', 'attr_6', 'attr_7', 'attr_8']


In [8]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder for each attribute column
label_encoders = {}
for column in attr_columns:
    le = LabelEncoder()
    train_df_Men_Tshirts[column] = le.fit_transform(train_df_Men_Tshirts[column])
    label_encoders[column] = le  # Store the encoder for inverse transformation later if needed

# Check the updated DataFrame
# train_df_Men_Tshirts = train_df_Men_Tshirts.head()

In [9]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and validation sets
train_df1, val_df1 = train_test_split(train_df_Men_Tshirts, test_size=0.15, random_state=42)

In [10]:
train_df1

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,image_path
5083,37684,Women Tshirts,8,3,2,2,0,0,2,1,1,/kaggle/input/visual-taxonomy/train_images/037...
11571,44172,Women Tshirts,8,5,2,0,1,1,2,1,1,/kaggle/input/visual-taxonomy/train_images/044...
7918,40519,Women Tshirts,8,1,2,2,1,3,2,1,1,/kaggle/input/visual-taxonomy/train_images/040...
17173,49774,Women Tshirts,8,5,2,0,1,5,2,1,1,/kaggle/input/visual-taxonomy/train_images/049...
17808,50409,Women Tshirts,8,5,2,2,1,5,2,1,1,/kaggle/input/visual-taxonomy/train_images/050...
...,...,...,...,...,...,...,...,...,...,...,...,...
11284,43885,Women Tshirts,8,5,2,0,1,1,2,1,1,/kaggle/input/visual-taxonomy/train_images/043...
11964,44565,Women Tshirts,8,5,2,2,1,3,2,1,1,/kaggle/input/visual-taxonomy/train_images/044...
5390,37991,Women Tshirts,8,6,2,2,1,5,1,1,1,/kaggle/input/visual-taxonomy/train_images/037...
860,33461,Women Tshirts,8,6,2,1,1,1,0,1,1,/kaggle/input/visual-taxonomy/train_images/033...


In [11]:
val_df1

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,image_path
11647,44248,Women Tshirts,8,5,2,0,1,3,2,1,1,/kaggle/input/visual-taxonomy/train_images/044...
13203,45804,Women Tshirts,8,6,2,2,1,5,2,1,1,/kaggle/input/visual-taxonomy/train_images/045...
1862,34463,Women Tshirts,8,5,1,1,1,2,0,1,1,/kaggle/input/visual-taxonomy/train_images/034...
2699,35300,Women Tshirts,8,6,2,2,0,0,2,1,1,/kaggle/input/visual-taxonomy/train_images/035...
18189,50790,Women Tshirts,8,5,2,0,1,5,2,1,1,/kaggle/input/visual-taxonomy/train_images/050...
...,...,...,...,...,...,...,...,...,...,...,...,...
420,33021,Women Tshirts,8,0,2,0,1,3,1,0,1,/kaggle/input/visual-taxonomy/train_images/033...
284,32885,Women Tshirts,8,3,2,2,0,0,1,0,1,/kaggle/input/visual-taxonomy/train_images/032...
998,33599,Women Tshirts,8,4,2,0,1,5,0,1,1,/kaggle/input/visual-taxonomy/train_images/033...
13251,45852,Women Tshirts,8,5,2,2,1,1,2,1,1,/kaggle/input/visual-taxonomy/train_images/045...


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import numpy as np

# Define custom dataset
class CustomDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, is_test=False):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test  # Flag to indicate if it's test set without labels

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.img_dir + self.dataframe.iloc[idx]['image_path']
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        if self.is_test:  # For test set, just return the image without labels
            return image
        
        # For train/validation set, return image and labels
        labels = self.dataframe.iloc[idx][attr_columns].values
        labels = labels.astype(np.int64)  # Ensure labels are integers
        labels = torch.tensor(labels, dtype=torch.long)
        
        return image, labels
    
    
# Transform
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Increased size for more detail
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Crop to focus on various areas
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(p=0.2),  # Add a slight chance of vertical flip for variety
    transforms.RandomRotation(15),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), shear=10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
    
train_dataset = CustomDataset(dataframe=train_df1, img_dir='', transform=transform)
val_dataset = CustomDataset(dataframe=val_df1, img_dir='', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [13]:
import ssl

ssl._create_default_https_context = ssl._create_stdlib_context

In [14]:
from torchvision import models
available_models = dir(models)

# Filter out private members and display model names
available_models = [model for model in available_models if not model.startswith('_')]
print("Available models in torchvision.models:")
print(available_models)

Available models in torchvision.models:
['AlexNet', 'AlexNet_Weights', 'ConvNeXt', 'ConvNeXt_Base_Weights', 'ConvNeXt_Large_Weights', 'ConvNeXt_Small_Weights', 'ConvNeXt_Tiny_Weights', 'DenseNet', 'DenseNet121_Weights', 'DenseNet161_Weights', 'DenseNet169_Weights', 'DenseNet201_Weights', 'EfficientNet', 'EfficientNet_B0_Weights', 'EfficientNet_B1_Weights', 'EfficientNet_B2_Weights', 'EfficientNet_B3_Weights', 'EfficientNet_B4_Weights', 'EfficientNet_B5_Weights', 'EfficientNet_B6_Weights', 'EfficientNet_B7_Weights', 'EfficientNet_V2_L_Weights', 'EfficientNet_V2_M_Weights', 'EfficientNet_V2_S_Weights', 'GoogLeNet', 'GoogLeNetOutputs', 'GoogLeNet_Weights', 'Inception3', 'InceptionOutputs', 'Inception_V3_Weights', 'MNASNet', 'MNASNet0_5_Weights', 'MNASNet0_75_Weights', 'MNASNet1_0_Weights', 'MNASNet1_3_Weights', 'MaxVit', 'MaxVit_T_Weights', 'MobileNetV2', 'MobileNetV3', 'MobileNet_V2_Weights', 'MobileNet_V3_Large_Weights', 'MobileNet_V3_Small_Weights', 'RegNet', 'RegNet_X_16GF_Weights', '

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

class MultiOutputModel1(nn.Module):
    def __init__(self, num_classes):
        super(MultiOutputModel1, self).__init__()
        # Use MobileNetV2 as the base model
        self.base_model = models.efficientnet_b6(pretrained=True)
        self.base_model.classifier = nn.Identity()  # Remove original classification layer
        
        # Add an adaptive pooling layer to make sure output is flat
        self.pooling = nn.AdaptiveAvgPool2d((1, 1))  # Convert 2D output to 1D
        
        # Dynamically create a fully connected layer for each attribute
        self.output_layers = nn.ModuleDict()
        for attr, n_classes in num_classes.items():
            self.output_layers[attr] = nn.Linear(2304, n_classes)  # Adjust input to 1280 for MobileNetV2

    def forward(self, x):
        x = self.base_model.features(x)  # Extract features
        x = self.pooling(x)  # Adaptive pool to (1, 1) shape
        x = torch.flatten(x, 1)  # Flatten the output to (batch_size, 1280)
        
        outputs = {}
        # Dynamically compute output for each attribute
        for attr, layer in self.output_layers.items():
            outputs[attr] = layer(x)
        
        return list(outputs.values())

# Example usage
num_classes = {}
for key in attr_columns:
    num_classes[key] = train_df_Men_Tshirts[key].nunique()

model = MultiOutputModel1(num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

Downloading: "https://download.pytorch.org/models/efficientnet_b6_lukemelas-24a108a5.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b6_lukemelas-24a108a5.pth
100%|██████████| 165M/165M [00:00<00:00, 201MB/s]


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Training loop
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, save_path=f"../../Models/Women Tshirts/eff_net_b6_fillna_eff_net_{model_category}_preproc.pth"):
    scheduler = optim.lr_scheduler.StepLR(optimizer = optimizer , step_size=5, gamma=0.5)
    best_val_loss = float('inf')
    for epoch in tqdm(range(num_epochs)):
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)  # Move tensors to the correct device
            optimizer.zero_grad()
            outputs = model(images)
            loss = sum([criterion(output, label) for output, label in zip(outputs, labels.T)])
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        scheduler.step()
        
        print(f'Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader)}')

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)  # Move tensors to the correct device
                outputs = model(images)
                loss = sum([criterion(output, label) for output, label in zip(outputs, labels.T)])
                val_loss += loss.item()
        
        avg_val_loss = val_loss / len(val_loader)
        print(f'Epoch {epoch+1}, Val Loss: {val_loss/len(val_loader)}')
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), save_path)
            print(f"Model saved with Val Loss: {best_val_loss:.4f} at Epoch {epoch+1}")

# Run training
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=7)

model.load_state_dict(torch.load(f"../../Models/Women Tshirts/eff_net_b6_fillna_eff_net_{model_category}_preproc.pth"))

  0%|          | 0/7 [00:00<?, ?it/s]

Epoch 1, Train Loss: 4.014086905366672
Epoch 1, Val Loss: 2.5716842372765703
Model saved with Val Loss: 2.5717 at Epoch 1
Epoch 2, Train Loss: 2.4683925954994552
Epoch 2, Val Loss: 2.238166660405277
Model saved with Val Loss: 2.2382 at Epoch 2
Epoch 3, Train Loss: 2.1175636334983046
Epoch 3, Val Loss: 2.0496145031425392
Model saved with Val Loss: 2.0496 at Epoch 3
Epoch 4, Train Loss: 1.9114310153739487
Epoch 4, Val Loss: 2.0286726723895985
Model saved with Val Loss: 2.0287 at Epoch 4
Epoch 5, Train Loss: 1.7463893050660113
Epoch 5, Val Loss: 2.026679727468598
Model saved with Val Loss: 2.0267 at Epoch 5
Epoch 6, Train Loss: 1.5136442049471792
Epoch 6, Val Loss: 1.930526840552855
Model saved with Val Loss: 1.9305 at Epoch 6
Epoch 7, Train Loss: 1.408529988988368
Epoch 7, Val Loss: 1.9996693375405301


<All keys matched successfully>